In [19]:
! pip install geopandas
! pip install pulp
! pip install spopt
! pip install glpk
! pip install coinor-cbc
! pip install matplotlib-scalebar

  Using cached glpk-0.4.7.tar.gz (161 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build glpk


  error: subprocess-exited-with-error
  
  × Building wheel for glpk (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [11 lines of output]
      WARNING setuptools_scm.pyproject_reading toml section missing 'pyproject.toml does not contain a tool.setuptools_scm section'
      Traceback (most recent call last):
        File "C:\Users\ASUS\AppData\Local\Temp\pip-build-env-46e9964i\overlay\Lib\site-packages\setuptools_scm\_integration\pyproject_reading.py", line 36, in read_pyproject
          section = defn.get("tool", {})[tool_name]
                    ~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^
      KeyError: 'setuptools_scm'
      running bdist_wheel
      running build
      running build_ext
      building 'glpk' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a p

In [20]:
%config InlineBackend.figure_format = "retina"
%load_ext watermark
%watermark

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Last updated: 2024-07-04T09:44:11.497710+07:00

Python implementation: CPython
Python version       : 3.12.3
IPython version      : 8.24.0

Compiler    : MSC v.1938 64 bit (AMD64)
OS          : Windows
Release     : 11
Machine     : AMD64
Processor   : Intel64 Family 6 Model 140 Stepping 1, GenuineIntel
CPU cores   : 8
Architecture: 64bit



In [21]:
import pandas as pd
import networkx as nx
import geopandas
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import matplotlib.lines as mlines
import matplotlib_scalebar
from matplotlib_scalebar.scalebar import ScaleBar
from shapely.geometry import Point
import time
import numpy as np
import pulp
import shapely
import spopt
from spopt.locate import PCenter, simulated_geo_points

import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    # ignore deprecation warning - GH pysal/spaghetti#649
    import spaghetti

%watermark -w
%watermark -iv

Watermark: 2.4.3

geopandas          : 0.14.4
shapely            : 2.0.4
networkx           : 3.3
matplotlib         : 3.9.0
pandas             : 2.2.2
pulp               : 2.8.0
matplotlib_scalebar: 0.8.1
spaghetti          : 1.7.5.post1
numpy              : 1.26.4
spopt              : 0.6.0



In [22]:
network_distance = pd.read_csv('C:/Users/ASUS/Downloads/rescue_station_project/scripts/test_demo/csv/network_distance.csv')
network_distance

,StartPoint,EndPoint,Distance
0,1,1,0
1,1,2,9
2,1,4,10
3,1,3,12
4,1,6,17
...,...,...,...
429,15,19,8
430,16,18,7
431,17,18,6
432,18,20,30


In [23]:
facility_points = pd.read_csv('C:/Users/ASUS/Downloads/rescue_station_project/scripts/test_demo/csv/facility_points.csv')
facility_points = facility_points.reset_index(drop=True)
facility_points

,Id,FacilityPoints,XX,YY
0,1,1,617714,1197576
1,2,2,617714,1196576
2,3,3,618714,1197076
3,4,4,619714,1197576
4,5,5,619714,1196576
5,6,6,620714,1197076
6,7,7,618714,1198076
7,8,8,619714,1198076
8,9,9,620714,1198076
9,10,10,619714,1198576


In [24]:
# Tạo bảng pivot từ dataframe
pivot_table = network_distance.pivot_table(values='Distance', index='StartPoint', columns='EndPoint', aggfunc=min)

# Thay thế các giá trị NaN = 0
cost_matrix = pivot_table.fillna(0).astype(int)
# cost_matrix[cost_matrix == 0] = 1

print(cost_matrix)


EndPoint    1   2   3   4   5   6   7   8   9   10  11  12  13  14  15  16  \
StartPoint                                                                   
1            0   9  12  10  19  17  25  23  30  35  25  33  39  45  31  43   
2            9   0  13  19  17  26  34  32  39  44  34  42  48  54  40  52   
3           12  13   0   8   7  15  31  21  28  41  23  31  37  51  29  41   
4           10  19   8   0  13   7  23  13  20  33  15  23  29  43  21  33   
5           19  17   7  13   0   9  36  26  33  46  28  36  42  56  23  46   
6           17  26  15   7   9   0  30  20  27  40  21  29  35  50  14  39   
7           25  34  31  23  36  30   0  10  25  10  38  41  35  20  44  51   
8           23  32  21  13  26  20  10   0  33  20  28  36  42  30  34  46   
9           30  39  28  20  33  27  25  33   0  15  24  16  10  40  31  26   
10          35  44  41  33  46  40  10  20  15   0  39  31  25  25  46  41   
11          25  34  23  15  28  21  38  28  24  39   0   8  14  

C:\Users\ASUS\AppData\Local\Temp\ipykernel_13960\622129948.py:2: FutureWarning: The provided callable <built-in function min> is currently using DataFrameGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  pivot_table = network_distance.pivot_table(values='Distance', index='StartPoint', columns='EndPoint', aggfunc=min)


In [25]:
# Nhập số lượng facilities cần tìm
p_facilities = int(input("Nhập số lượng facilities cần tìm: "))

# Số lượng điểm
num_points = cost_matrix.shape[0]

# Thiết lập bài toán quy hoạch nguyên
model = pulp.LpProblem("p-Center Problem", pulp.LpMinimize)

# Biến quyết định
x = pulp.LpVariable.dicts("x", (range(num_points), range(num_points)), 0, 1, pulp.LpBinary)
y = pulp.LpVariable.dicts("y", range(num_points), 0, 1, pulp.LpBinary)

# Hàm mục tiêu: minimize max distance
z = pulp.LpVariable("z", 0)
model += z

# Ràng buộc: mỗi điểm phải được phục vụ bởi đúng một facility
for i in range(num_points):
    model += pulp.lpSum(x[i][j] for j in range(num_points)) == 1

# Ràng buộc: số lượng facilities được chọn bằng p_facilities
model += pulp.lpSum(y[j] for j in range(num_points)) == p_facilities

# Ràng buộc: x[i][j] <= y[j] và z >= d_ij * x[i][j]
for i in range(num_points):
    for j in range(num_points):
        model += x[i][j] <= y[j]
        model += z >= cost_matrix.iloc[i, j] * x[i][j]

# Giải bài toán
model.solve()

# In kết quả
print(f"Khoảng cách tối ưu: {pulp.value(z)}")
print("Các điểm được chọn làm facility:")

selected_facilities = []
for j in range(num_points):
    if pulp.value(y[j]) == 1:
        facility_info = facility_points.iloc[j]
        selected_facilities.append({
            "Id": facility_info["Id"],
            "Facilities": facility_info["FacilityPoints"],
            "XX": facility_info["XX"],
            "YY": facility_info["YY"]
        })

# Tạo DataFrame từ danh sách các điểm facility được chọn
selected_facilities_df = pd.DataFrame(selected_facilities)
print(selected_facilities_df)

Khoảng cách tối ưu: 43.0
Các điểm được chọn làm facility:
   Id  Facilities      XX       YY
0   9           9  620714  1198076


In [26]:
import pandas as pd

# Tạo bảng pivot từ dataframe
pivot_table = network_distance.pivot_table(values='Distance', index='EndPoint', columns='StartPoint', fill_value=0)

# Chuyển đổi thành ma trận numpy để sử dụng với spopt và pulp
cost_matrix = pivot_table.values.astype(int)

In [37]:
# Nhập số lượng facilities cần tìm từ người dùng
p_facilities = int(input("Nhập số lượng facilities cần tìm: "))

# Khởi tạo và giải quyết bài toán LSCP
pcenter = PCenter.from_cost_matrix(cost_matrix, p_facilities)
pcenter = pcenter.solve(pulp.PULP_CBC_CMD(msg=False))

# Lấy giá trị objective
pcenter_objval = pcenter.problem.objective.value()

# Tạo danh sách các điểm được chọn trong pcenter_objval
selected_facilities = [i for i, dv in enumerate(pcenter.fac_vars) if dv.varValue]

# Tạo DataFrame từ danh sách các điểm được chọn
selected_facilities_df = facility_points.iloc[selected_facilities].reset_index(drop=True)

# In giá trị objective và DataFrame các điểm được chọn
print("Khoảng cách tối ưu: ", pcenter_objval)
print("Các điểm được chọn làm facility:")
print(selected_facilities_df)


Khoảng cách tối ưu:  16.0
Các điểm được chọn làm facility:
   Id  FacilityPoints      XX       YY
0   1               1  617714  1197576
1   7               7  618714  1198076
2  12              12  621714  1198076
3  14              14  617714  1198576
4  19              19  621714  1196576
